<a href="https://colab.research.google.com/github/AleAguirreM/LEA3_Health/blob/main/c_modelos_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import joblib ### para cargar array

########Paquetes para NN #########
import tensorflow as tf
from sklearn import metrics ### para analizar modelo
from sklearn.ensemble import RandomForestClassifier  ### para analizar modelo
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import keras

In [13]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [5]:
### cargar bases_procesadas ####

x_train = joblib.load('/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/x_train.pkl')
y_train = joblib.load('/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/y_train.pkl')
x_test = joblib.load('/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/x_test.pkl')
y_test = joblib.load('/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/y_test.pkl')


In [ ]:
#### Escalar ######################
x_train=x_train.astype('float32') ## para poder escalarlo
x_test=x_test.astype('float32') ## para poder escalarlo
x_train /=255 ### escalaro para que quede entre 0 y 1
x_test /=255

In [6]:
###### verificar tamaños

print(x_train.shape)
x_test.shape


(3360, 100, 100, 3)


(840, 100, 100, 3)

In [7]:
np.product(x_train[1].shape) ## cantidad de variables por imagen

30000

In [9]:
print(np.unique(y_train, return_counts=True))
np.unique(y_test, return_counts=True)

(array([0, 1]), array([2800,  560]))


(array([0, 1]), array([700, 140]))

In [10]:
##########################################################
################ Redes convolucionales ###################
##########################################################

cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[1:]),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with binary cross-entropy loss and Adam optimizer
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

In [11]:
# Train the model for 10 epochs
cnn_model.fit(x_train, y_train, batch_size=100, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
34/34 [==============================] - 28s 770ms/step - loss: 13.9474 - auc: 0.7308 - val_loss: 2.3545 - val_auc: 0.5428
Epoch 2/10
34/34 [==============================] - 23s 683ms/step - loss: 0.3584 - auc: 0.9326 - val_loss: 0.3285 - val_auc: 0.9378
Epoch 3/10
34/34 [==============================] - 22s 649ms/step - loss: 0.1234 - auc: 0.9796 - val_loss: 0.1843 - val_auc: 0.9765
Epoch 4/10
34/34 [==============================] - 23s 685ms/step - loss: 0.0831 - auc: 0.9900 - val_loss: 0.1930 - val_auc: 0.9799
Epoch 5/10
34/34 [==============================] - 21s 636ms/step - loss: 0.0563 - auc: 0.9935 - val_loss: 0.1138 - val_auc: 0.9849
Epoch 6/10
34/34 [==============================] - 28s 834ms/step - loss: 0.0461 - auc: 0.9964 - val_loss: 0.1823 - val_auc: 0.9812
Epoch 7/10
34/34 [==============================] - 25s 743ms/step - loss: 0.0393 - auc: 0.9965 - val_loss: 0.1040 - val_auc: 0.9855
Epoch 8/10
34/34 [==============================] - 23s 679ms/step -

In [14]:
#######probar una red con regulzarización


#####################################################
###### afinar hiperparameter ########################
#####################################################
####instalar paquete !pip install keras-tuner
import keras_tuner as kt

In [15]:
##### función con definicion de hiperparámetros a afinar

def build_model(hp):

    dropout_rate=hp.Float('DO', min_value=0.1, max_value= 0.4, step=0.05)
    reg_strength = hp.Float("rs", min_value=0.0001, max_value=0.0005, step=0.0001)
    ####hp.Int
    ####hp.Choice


    model=tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=x_train.shape[1:]),
        tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=0.01)
    else:
        opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

    model.compile(
        optimizer=opt, loss="binary_crossentropy", metrics=["AUC"],
    )
    return model


In [16]:
###########
hp = kt.HyperParameters()
build_model(hp)

tuner = kt.RandomSearch(
    hypermodel=build_model,
    hyperparameters=hp,
    tune_new_entries=False, ## solo evalúe los hiperparámetros configurados
    objective=kt.Objective("val_auc", direction="max"),
    max_trials=10,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

tuner.search(x_train, y_train, epochs=3, validation_data=(x_test, y_test), batch_size=100)

fc_best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()

Trial 10 Complete [00h 00m 11s]
val_auc: 0.895959198474884

Best val_auc So Far: 0.9011836647987366
Total elapsed time: 00h 01m 59s
Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_auc", direction="max")

Trial 01 summary
Hyperparameters:
DO: 0.30000000000000004
rs: 0.0004
optimizer: adam
Score: 0.9011836647987366

Trial 09 summary
Hyperparameters:
DO: 0.15000000000000002
rs: 0.0002
optimizer: rmsprop
Score: 0.895959198474884

Trial 06 summary
Hyperparameters:
DO: 0.1
rs: 0.0005
optimizer: rmsprop
Score: 0.8792295455932617

Trial 00 summary
Hyperparameters:
DO: 0.1
rs: 0.0004
optimizer: rmsprop
Score: 0.8749693632125854

Trial 07 summary
Hyperparameters:
DO: 0.25
rs: 0.0004
optimizer: rmsprop
Score: 0.8472855687141418

Trial 02 summary
Hyperparameters:
DO: 0.15000000000000002
rs: 0.00030000000000000003
optimizer: rmsprop
Score: 0.8338317275047302

Trial 05 summary
Hyperparameters:
DO: 0.25
rs: 0.0001
optimizer: sgd
Score: 0.5007143020629883

Trial

In [18]:
#################### Mejor redes ##############

joblib.dump(fc_best_model, "/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/fc_model.pkl")
joblib.dump(cnn_model, "/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/cnn_model.pkl")

['/content/drive/MyDrive/TB_Chest_Radiography_Database/salidas/cnn_model.pkl']